<a href="https://colab.research.google.com/github/flyingsaucer123/UPC_product_search/blob/main/webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain-openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Succes

In [ ]:
!pip install crewai langchain-google-genai crewai_tools



  Using cached crewai-0.61.0-py3-none-any.whl.metadata (15 kB)
  Using cached langchain_google_genai-2.0.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached crewai_tools-0.12.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached auth0_python-4.7.2-py3-none-any.whl.metadata (8.9 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is co

In [ ]:
# Import necessary libraries
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from langchain_openai import ChatOpenAI  # Updated import
from dotenv import load_dotenv
import os
load_dotenv()

# Set up your OpenAI API key
openai_api_key ="open_ai_key"
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_MODEL_NAME"]="gpt-4-0125-preview"
os.environ["OPENAI_API_KEY"] = openai_api_key
# Setting up ChatOpenAI with the API key
llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # Setting model to GPT-3.5 Turbo
    temperature=0.5,
    openai_api_key=openai_api_key,  # Pass in your OpenAI API key here
    verbose=True
)

# Set up the Serper API key
os.environ["SERPER_API_KEY"] = "serper_key"  # Serper API Key

# Define the Serper search tool and ScrapeWebsiteTool
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [ ]:

# Define the UPC Search Agent
upc_search_agent = Agent(
    role="UPC Search Specialist",
    goal="Search for the UPC code based on the product description, manufacturer, and part number.",
    backstory="You are skilled at retrieving UPC codes based on product data from various online sources.",
    tools=[search_tool],
    allow_delegation=True,
    llm=llm,  # Using ChatOpenAI for language modeling
    verbose=True,
    memory=True
)

# Define the Scraping Agent
scraping_agent = Agent(
    role="Scraping Expert",
    goal="Scrape relevant web pages to extract UPC codes from search results.",
    backstory="You specialize in scraping and extracting data from websites.",
    tools=[scrape_tool],
    allow_delegation=False,
    llm=llm,  # Using ChatOpenAI for language modeling
    verbose=True,
    memory=True
)

# Refined UPC Search Task
upc_search_task = Task(
    description=(
        "Use the product details (manufacturer: {manufacturer_name}, description: {product_description}, part number: {manufacturer_part_number}) "
        "to search the web for a matching UPC code. Use all the product details for accurate search results."
    ),
    expected_output="List of relevant web links from search results.",
    tools=[search_tool],
    agent=upc_search_agent
)

# Refined Scraping Task to scrape the web pages for UPC codes
upc_scraping_task = Task(
    description=(
        "Scrape the relevant web pages found during the search for UPC codes based on the search results. "
        "Make sure to extract the UPC code or related product information from the scraped data."
    ),
    expected_output="Scraped content that contains the UPC code or related product information.",
    tools=[scrape_tool],  # Scraping the web pages returned from search
    agent=scraping_agent
)

# Create the crew for the UPC retrieval and scraping process
crew = Crew(
    agents=[upc_search_agent, scraping_agent],
    tasks=[upc_search_task, upc_scraping_task],
    process=Process.sequential,  # Running the tasks in sequence: Search -> Scrape
    verbose=True,
    memory=True
)

# Example product data to search for UPC
inputs = {
    "manufacturer_name": "ABCO PRODUCTS",
    "product_description": "32 GAL ROUND WASTE RECEPTACLE",
    "manufacturer_part_number": "BS32G"
}

# Run the crew
result = crew.kickoff(inputs=inputs)

# Display the final result
print(result)


# Agent: UPC Search Specialist
## Task: Use the product details (manufacturer: ABCO PRODUCTS, description: 32 GAL ROUND WASTE RECEPTACLE, part number: BS32G) to search the web for a matching UPC code. Use all the product details for accurate search results.


# Agent: UPC Search Specialist
## Thought: I should use the tool "Search the internet" to search for the UPC code based on the manufacturer, description, and part number provided for the waste receptacle by ABCO PRODUCTS.
## Using tool: Search the internet
## Tool Input: 
{
  "search_query": "ABCO PRODUCTS 32 GAL ROUND WASTE RECEPTACLE BS32G UPC code"
}
## Tool Output: 

Search results: Title: Round Waste Receptacles and Lids - ABCO Cleaning Products
Link: https://abcoproducts.com/products/waste-cans/waste-cans-and-accessories/round-waste-receptacles-and-lids
Snippet: Round Waste Receptacles and Lids ; BS-32G, 32 gallon, Grey, 51.72. ADD TO QUOTE ; BS-44G, 44 gallon, Grey, 42.04. ADD TO QUOTE.
---
Title: Dolly for Round Waste Rece

In [ ]:

# Define the UPC Search Agent
upc_search_agent = Agent(
    role="UPC Search Specialist",
    goal="Search for the UPC code based on the product description, manufacturer, and part number.",
    backstory="You are skilled at retrieving UPC codes based on product data from various online sources.",
    tools=[search_tool],
    allow_delegation=True,
    verbose=True,
    memory=True
)

# Define the Scraping Agent
scraping_agent = Agent(
    role="Scraping Expert",
    goal="Scrape relevant web pages to extract UPC codes from search results.",
    backstory="You specialize in scraping and extracting data from websites.",
    tools=[scrape_tool],
    allow_delegation=False,
    verbose=True,
    memory=True
)

# Refined UPC Search Task
upc_search_task = Task(
    description=(
        "Use the product details (manufacturer: {manufacturer_name}, description: {product_description}, part number: {manufacturer_part_number}) "
        "to search the web for a matching UPC code. Use all the product details for accurate search results."
    ),
    expected_output="List of relevant web links from search results.",
    tools=[search_tool],
    agent=upc_search_agent
)

# Refined Scraping Task to scrape the web pages for UPC codes
upc_scraping_task = Task(
    description=(
        "Scrape the relevant web pages found during the search for UPC codes based on the search results. "
        "Make sure to extract the UPC code or related product information from the scraped data."
    ),
    expected_output="Scraped content that contains the UPC code or related product information.",
    tools=[scrape_tool],  # Scraping the web pages returned from search
    agent=scraping_agent
)

# Create the crew for the UPC retrieval and scraping process
crew = Crew(
    agents=[upc_search_agent, scraping_agent],
    tasks=[upc_search_task, upc_scraping_task],
    process=Process.sequential,  # Running the tasks in sequence: Search -> Scrape
    verbose=True,
    memory=True
)

# Example product data to search for UPC
inputs = {
    "manufacturer_name": "Trader joes",
    "product_description": "pound milk chocolate",
    "manufacturer_part_number": ""
}

# Run the crew with the provided inputs
result = crew.kickoff(inputs=inputs)

# Display the final result
print(result)


# Agent: UPC Search Specialist
## Task: Use the product details (manufacturer: Trader joes, description: pound milk chocolate, part number: ) to search the web for a matching UPC code. Use all the product details for accurate search results.


# Agent: UPC Search Specialist
## Thought: Thought: I will use the provided product details to search the internet for the matching UPC code.
## Using tool: Search the internet
## Tool Input: 
{
  "search_query": "Trader Joe's pound milk chocolate UPC code"
}
## Tool Output: 

Search results: Title: Pound Plus Milk Chocolate Bar | Trader Joe's
Link: https://www.traderjoes.com/home/products/pdp/pound-plus-milk-chocolate-bar-020468
Snippet: Why? Because they're more than a pound of delicious, creamy, dreamy chocolate. In this case, it's Milk Chocolate with 33% cocoa solids, neatly packaged and easy ...
---
Title: Pound Plus Milk Chocolate Bar with Almonds - Trader Joe's
Link: https://www.traderjoes.com/home/products/pdp/pound-plus-milk-chocolate-ba